In [44]:
import cv2
import numpy as np
import os
import os.path
import matplotlib.pyplot as plt

# Load gambar dari folder dan memca gambar 

In [67]:
#menentukan direktori/folder data citra yang akan dibuka
lokasi_gambar = '../image/dataset/'

#menentukan ukuran tinggi dan lebar gambar
tinggi = 200
lebar = 200
dim = (lebar,tinggi)

tampungan = []
x = []

#mengumpulkan data citra yang akan dibuka dalam satu array
for path,sub_directory,files in os.walk(lokasi_gambar):
    for name in files:
        #Gabungkan nama file gambar dengan lokasi gambar
        img_path = (os.path.join(path,name))
        if img_path.endswith("png"):
            #Membaca gambar dari img_path
            img = cv2.imread(img_path)
            #preprocessing data / segentasi  boleh dilakukan
            convert = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            resized=cv2.resize(convert,dim, interpolation=cv2.INTER_LINEAR)
            blur = cv2.GaussianBlur(resized,(5,5),0)
#             print(blur.shape)
            tampungan.append(blur.flatten())
x = np.array(tampungan)
print(x.shape)

(184, 120000)


# Labeling

In [68]:
y0 = np.zeros(114)
y1 = np.ones(70)
y = []
y = np.concatenate((y0,y1), axis=0)
print(y)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [69]:
from sklearn.model_selection import GridSearchCV, train_test_split #library untuk train test split

#melakukan splitting data
X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.20, stratify=y) #train size adalah persentase data test yang di-split

from builtins import range
from builtins import object
num_training = X_train.shape[0] #deklarasi untuk data training diambil dari X_train

mask = list(range(num_training))
X_train = X_train[mask]
y_train = y_train[mask]

num_test = X_test.shape[0] #deklarasi untuk data diambil dari Y_train
mask = list(range(num_test))
X_test = X_test[mask]
y_test = y_test[mask]

# Mengubah data citra menjadi baris
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))

print("X_train: "+str(X_train.shape))
print("X_test: "+str(X_test.shape))
print("y_train: "+str(y_train.shape))
print("y_test: "+str(y_test.shape))

X_train: (147, 120000)
X_test: (37, 120000)
y_train: (147,)
y_test: (37,)


In [70]:
from sklearn import svm
from sklearn.model_selection import cross_validate, cross_val_score

# membangun model SVM
model = svm.SVC(C=1.0, kernel="linear" ,decision_function_shape='ovr', probability=True)
score = cross_validate(model, X_train, y_train, scoring='accuracy',
                         cv=5, return_train_score=True)
print(score['test_score'].mean())

0.9250574712643678


In [43]:
from sklearn.metrics import classification_report

# melakukan uji data
model.fit(X_train, y_train )
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.86      1.00      0.92        12
         1.0       1.00      0.82      0.90        11

    accuracy                           0.91        23
   macro avg       0.93      0.91      0.91        23
weighted avg       0.93      0.91      0.91        23



In [74]:
import matplotlib.pyplot as plt
img = cv2.imread('../image/dataset/telur/opencv_frame_3.png')

#Hasil Uji Klasifikasi pada dataset
Categories = ['telur_asin','Bukan']

# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_resize = cv2.resize(img,dim)
l = [img_resize.flatten()]
model.fit(x,y)
# model.fit(X_train, y_train)

# Mengecek hasil klasifikasi pada salah satu dataset
probability = model.predict_proba(l)
id_= int(model.predict(l)[0]) #prediksi
name = Categories[id_]+' -> '+ str(probability[0][id_]) #tulisan buat hasil
if probability.all() >= 0.5:
    print(name)
    cv2.imshow('Predikri',img)
cv2.waitKey(0)  
# menutup semua jendela yang terbuka
cv2.destroyAllWindows() 

Bukan -> 0.6497029451095907
